<a href="https://colab.research.google.com/github/DIPANJAN001/Forecasting-Solar-Energy/blob/master/bestresult06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install Boruta
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import concatenate
from keras import Model
from keras.layers import Input
from keras.layers import Bidirectional
from keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.layers import Input
from sklearn.decomposition import PCA 
from sklearn.metrics import mean_absolute_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [79]:
def lstm_data_transform(x_data, y_data, num_steps):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [80]:
df=pd.read_excel("/content/pv_06.xlsx")
weather_input1=df.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
solpow=df['power_normed']

In [81]:
rfc = RandomForestRegressor(random_state=1, n_estimators=1000, max_depth=7)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1)
boruta_selector.fit(np.array(weather_input), np.array(solpow)) 

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	33
Iteration: 	9 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	33
Iteration: 	10 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	33
Iteration: 	11 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	33
Iteration: 	12 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	33
Iteration: 	13 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	33
Iteration: 	14 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	33
Iteration: 	15 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	33
Iteration: 	16 / 100
Confirmed: 	10
Tentative: 	5
Rejected: 

BorutaPy(estimator=RandomForestRegressor(max_depth=7, n_estimators=72,
                                         random_state=RandomState(MT19937) at 0x7FAAF3BC9440),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FAAF3BC9440, verbose=2)

In [82]:
X_important_train = boruta_selector.transform(np.array(weather_input))
num_steps = 3
# training set
(x_transformed_train,
 y_transformed_train) = lstm_data_transform(X_important_train,solpow , num_steps=num_steps)
assert x_transformed_train.shape[0] == y_transformed_train.shape[0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_transformed_train,y_transformed_train,test_size=0.25, random_state=42,shuffle=False)
#X_train_,X_val,y_train_,y_val=train_test_split(X_train,y_train,test_size=0.2, random_state=42,shuffle=False)
inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))

In [83]:
inputs1

<KerasTensor: shape=(None, 3, 11) dtype=float32 (created by layer 'input_4')>

In [84]:
from keras import optimizers

In [85]:
opt=optimizers.Adam(learning_rate=0.003)

In [86]:
from keras.layers import Bidirectional

In [87]:
import tensorflow as tf

In [88]:
from tensorflow.keras.optimizers import Adam
from tensorflow.python.ops import math_ops

class HalvAdam(Adam):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.prev_gradients = None

    @tf.function
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [math_ops.cast(x, "float32") for x in grads]

        if self.prev_gradients is not None:
            for i in range(len(grads)):
                if (grads[i] * self.prev_gradients[i] < 0):
                    self.updates[i] = self.updates[i] / 2

        self.prev_gradients = grads
        return self.updates

In [89]:
from keras import backend as K

In [90]:
import tensorflow as tf

In [91]:
def define_model():


  fe2_0 = Bidirectional(LSTM(256, activation='LeakyReLU',return_sequences = True))(inputs1)
  fe2_1 = Dropout(0.6)(fe2_0)
  fe2_2 = Bidirectional(LSTM(64, activation='LeakyReLU',return_sequences = True))(fe2_1)
  fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=Bidirectional(LSTM(4, activation='LeakyReLU'))(fe2_3)
  out2_1=Dense(1, activation='relu')(fe2_4)

  fe3_0 =Bidirectional(LSTM(128, activation='LeakyReLU',return_sequences = True))(inputs1)
  fe3_1 = Dropout(0.6)(fe3_0)
  fe3_2 = Bidirectional(LSTM(96, activation='LeakyReLU',return_sequences = True))(fe3_1)
  fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=Bidirectional(LSTM(8, activation='LeakyReLU'))(fe3_3)#16
  out3_1=Dense(1, activation='relu')(fe3_4)
 
 

  output = layers.average([out2_1, out3_1])
  #merged3 = concatenate([out2_1,out3_1], name='concat3')
  #output = Dense(1, activation='relu')( merged3)
  

  model = Model(inputs=[inputs1], outputs=[output])
  
 
  return model
mdl=define_model()


In [92]:
loss=[]

In [93]:
from keras.models import clone_model

In [94]:
from keras.optimizers import Adam

In [95]:
from keras.callbacks import ModelCheckpoint

In [96]:
def GlobalMinimaSearch(weights):
  if len(loss)>4:
   return
  
  initial_weights =weights
  model=clone_model(mdl)
  model.set_weights(weights)
  model.compile(optimizer=HalvAdam(learning_rate=0.003), loss='mean_squared_error')
  model.fit(X_train, y_train, epochs=120, batch_size=128)
  y= model.predict(X_test)
  loss.append(mean_absolute_error(y,y_test))
  best_weights= model.get_weights()
  

     

  params_1 =[final_weight + (final_weight - initial_weight) for initial_weight, final_weight in zip(initial_weights, best_weights)]
  #GlobalMinimaSearch(params_1)


  params_2 =[final_weight - (final_weight - initial_weight) for initial_weight, final_weight in zip(initial_weights, best_weights)]
  GlobalMinimaSearch(params_2)
  
 

In [97]:
GlobalMinimaSearch(mdl.get_weights())

Epoch 1/120
30/30 [==============================] - 25s 166ms/step - loss: 0.0152
Epoch 2/120
30/30 [==============================] - 3s 115ms/step - loss: 0.0094
Epoch 3/120
30/30 [==============================] - 3s 115ms/step - loss: 0.0078
Epoch 4/120
30/30 [==============================] - 5s 165ms/step - loss: 0.0078
Epoch 5/120
30/30 [==============================] - 4s 131ms/step - loss: 0.0076
Epoch 6/120
30/30 [==============================] - 4s 146ms/step - loss: 0.0072
Epoch 7/120
30/30 [==============================] - 5s 160ms/step - loss: 0.0074
Epoch 8/120
30/30 [==============================] - 4s 116ms/step - loss: 0.0071
Epoch 9/120
30/30 [==============================] - 4s 129ms/step - loss: 0.0073
Epoch 10/120
30/30 [==============================] - 5s 168ms/step - loss: 0.0068
Epoch 11/120
30/30 [==============================] - 3s 116ms/step - loss: 0.0068
Epoch 12/120
30/30 [==============================] - 3s 115ms/step - loss: 0.0066
Epoch 13/120

KeyboardInterrupt: ignored

In [98]:
print(loss)

[0.03447041984061651, 0.03258155936356708, 0.034770220111454954, 0.031425704986059746]


In [99]:
print(min(loss))

0.031425704986059746


In [100]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(loss)